## Evaluation
- Random Initialize Model
- spherical005.nc 데이터 대상으로 평가

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import time
import os
import numpy as np
import matplotlib.pyplot as plt

from dataset import EarthMantleDataset
from dataset import read_cdf

from resnet import ResNet
from resnet import Bottleneck
from resnet import BasicBlock

In [2]:
target_dir = 'D:/EarthMantleConvection/results/'

In [3]:
file_path = 'D:/EarthMantleConvection/mantle01/spherical005.nc'
x_volume, y_volume, volume_size = read_cdf(file_path, 5, 5, 5, './scalers')
eval_set = EarthMantleDataset(x_volume, y_volume, volume_size, file_path)
eval_loader = data.DataLoader(dataset=eval_set, batch_size=1024, num_workers=2, shuffle=False)

In [4]:
print(eval_set)
print(len(eval_set))
print(eval_set.get_volume_size())
print(eval_set.get_x_volume_size())
print(eval_set.get_y_volume_size())
print(eval_set.in_channels)
print(eval_set.depth)
print(eval_loader.batch_size)

13024800
(201, 180, 360)
torch.Size([7, 205, 184, 364])
torch.Size([1, 205, 184, 364])
7
5
1024


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
model = ResNet(Bottleneck, [1, 1, 1, 1], [16, 32, 64, 128], eval_set.in_channels, out_dim=eval_set.depth)
model.to(device)
model.eval()

Device: cuda


ResNet(
  (conv1): Conv3d(7, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (block1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(16, 16, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(16, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv3d(16, 64, kern

In [6]:
do = eval_set.depth // 2
ho = eval_set.height // 2
wo = eval_set.width // 2

dl = do
du = do + 1

s = time.time()

res = np.zeros((eval_set.get_y_volume_size()[1:]))
cnt = np.zeros((eval_set.get_y_volume_size()[1:]))
n_data = len(eval_set)
j = 0
with torch.no_grad():
    for x, y, _idx in eval_loader:
        ss = time.time()
        x = x.to(device)
        y = y.to(device)
        idx = torch.stack(_idx, dim=1)
        # print(x.shape)
        # print(y.shape)
        # print(idx.shape)
        output = np.array(model(x).cpu())
        # print(output.shape)
        for i in range(idx.shape[0]):
            ix, iy, iz = idx[i, :]
            res[ix-dl:ix+du, iy, iz] += output[i, :]
            cnt[ix-dl:ix+du, iy, iz] += 1
        print(f'\rEvaluation: {j}/{n_data}, Step Time: {time.time()-ss:.2f}', end='')
        j += eval_loader.batch_size

    print()
print(f'Elapsed Time: {time.time() - s:.2f}')

res = res[do:-do, ho:-ho, wo:-wo] / cnt[do:-do, ho:-ho, wo:-wo]

os.makedirs(target_dir, exist_ok=True)

data_file = os.path.basename(eval_set.netcdf_file).split('.')[0]
res_path = os.path.join(target_dir, f'Res_{data_file}_version_01_RandomInit.npy')

np.save(res_path, res)
print(res_path)

Evaluation: 13024256/13024800, Step Time: 0.03
Elapsed Time: 570.79
D:/EarthMantleConvection/results/Res_spherical005_version_01_RandomInit.npy
